# UnionML Demo

In [ ]:
from pictionary_app import model


model.remote_deploy()

In [2]:
from pictionary_app import model

num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
    app_version='71ea176253225b67d5f9e3e2a59354913ffd85f0',
)

Executing quickdraw_classifier.train, execution name: fed4d7985216a4f0ead7.
Go to https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/fed4d7985216a4f0ead7 to see the execution in the console.


# 🚨🚨🚨 **NOTE FOR UNION.AI TEAM** 🚨🚨🚨 The following other settings will hopefully be cached and good
to go for use in the demo. You can try it out, and replace the code snippet above with one of the ones below.

### More labels:

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f27171bd39bec4f04929?duration=all

```python
num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### More labels, trained for longer

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f444bd69e68c747a59cf?duration=all

```python
num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 20},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### All labels, trained for longer

- cpu execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f23f637d2f3b74460a6f?duration=all
- gpu execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f03bd4bc57adf4db1b46?duration=all

```python
num_classes = 345

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 20},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### All labels, trained for longer much longer on gpu

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/fe652be0db07b473383e?duration=all

```python
num_classes = 345

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 45},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

In [ ]:
# wait for the execution to complete, then load model from remote
model.remote_load(execution)

In [ ]:
import gradio as gr

gr.Interface(
    fn=model.predict,
    inputs="sketchpad",
    outputs="label",
    live=True,
    allow_flagging="never",
).launch()

# Flytekit Demo

In [ ]:
import torch.nn
from typing import List
from flytekit import workflow

from flytekit_demo.batch_predictions import batch_predictions_task, prepare_map_inputs, download_quickdraw_dataset, generate_input, download_quickdraw_class_names

@workflow
def batch_predict(
    model_object: torch.nn.Module,
    n_entries: int,
    max_items_per_class: int = 1000,
    num_classes: int = 200,
) -> (List[dict], List[torch.Tensor], List[str]):
    class_names = download_quickdraw_class_names()
    dataset = download_quickdraw_dataset(max_items_per_class=max_items_per_class, num_classes=num_classes)
    feature_list, label_list = generate_input(n_entries=n_entries, dataset=dataset, class_names=class_names)
    map_input = prepare_map_inputs(model_object=model_object, feature_list=feature_list)
    predictions = batch_predictions_task(input=map_input)
    return predictions, feature_list, label_list

Create a `FlyteRemote` object and fetch the batch prediction workflow

In [ ]:
from flytekit.remote import FlyteRemoproject=om flytekit.configuration import Config, SerializationSettings, ImageConfig

remote = FlyteRemote(config=Config.auto(config_file="./config/config-remote.yaml"), 
                     data_upload_location="s3://open-compute-playground/data",
                     default_project="unionml",
                     default_domain="development")
image_config = ImageConfig.auto(img_name="ghcr.io/unionai-oss/unionml:quickdraw-classifier-71ea176253225b67d5f9e3e2a59354913ffd85f0")
wf = remote.register_workflow(batch_predict, 
                         version="71ea176253225b67d5f9e3e2a59354913ffd85f0", 
                         serialization_settings=SerializationSettings(image_config=image_config))

In [ ]:
# NOTE: make sure you have aws credentials for the s3://open-compute-playground bucket
execution = remote.execute(
    wf,
    inputs={"model_object": model.artifact.model_object, "feature_list": feature_list},
    project="unionml",
    domain="development",
    wait=True,
)

In [ ]:
# NOTE: make sure you have aws credentials for the s3://open-compute-playground bucket
execution = remote.execute(
    wf,
    inputs={"model_object": model.artifact.model_object, "n_entries": 20, "max_items_per_class": 1000, "num_classes": 200},
    project="unionml",
    domain="development",
    wait=True,
)

In [ ]:
import math
from matplotlib import pyplot as plt

predictions = execution.outputs.get('o0')
feature_list = execution.outputs.get('o1')
label_list = execution.outputs.get('o2')

# Arrange results in a square where each image contains the prediction alongside with the original label
fig = plt.figure(figsize=(40, 40))

sqr = math.sqrt(len(predictions))
rows = math.ceil(sqr)
columns = math.floor(sqr) + 1

for i in range(len(predictions)):
    best_prediction = max(predictions[i], key=predictions[i].get)
    expected_label = label_list[i]

    fig.add_subplot(rows, columns, i + 1)
    plt.imshow(feature_list[i], interpolation='bicubic')
    plt.axis('off')
    plt.title(f"predicted: {best_prediction}\nexpected: {expected_label}")